In [3]:
from pymongo import MongoClient
from pprint import pprint

In [4]:
client = MongoClient("mongodb://10.177.185.218:11011/")

In [5]:
db = client["code-browser"]

In [6]:
collection = db["240718-datalake-snapshot"]

In [7]:
# 1.
pipeline = [
{
        "$project": {
            "source": 1,
            "system_prompt": { "$arrayElemAt": 
                [
                    {"$getField": {
                        "field": "candidates",
                        "input": { "$arrayElemAt": ["$data", 1] },
                    }},
                    0
                ]
            },
        },
        
    },
    {
        "$addFields": {
            "is_date_cutoff": {
                "$regexMatch": {
                    "input": "$system_prompt",
                    "regex": '.*오늘은 \\d+\\S \\d+\\S \\d+\\S.*'
                }
            }
        }
    },
    {
        "$project":{
            "source": 1,
            "system_prompt_type": {
                "$cond": {
                    "if": "$is_date_cutoff",
                    "then": "date_cutoff",
                    "else": "$system_prompt"
                }
            },
            "system_prompt": 1
        }
    },
    {
        "$group": {
            "_id": {
                "source": "$source",
                # "system_prompt_type": "$system_prompt_type"
            },
            "count": { "$sum": 1 },
        }
    },
]


In [8]:
list(collection.aggregate(pipeline))

[{'_id': {'source': 'autobrowsing'}, 'count': 726},
 {'_id': {'source': 'evalstyle_question_2'}, 'count': 233},
 {'_id': {'source': 'programmers'}, 'count': 75},
 {'_id': {'source': 'backjun_round2'}, 'count': 618},
 {'_id': {'source': 'aihub.150.numeric_mr'}, 'count': 69},
 {'_id': {'source': 'ci_seed'}, 'count': 420},
 {'_id': {'source': 'chart-to-table'}, 'count': 119},
 {'_id': {'source': '240613_error_correction'}, 'count': 95},
 {'_id': {'source': 'python_execution'}, 'count': 45},
 {'_id': {'source': 'aihub.71565.tableqa'}, 'count': 105},
 {'_id': {'source': 'unknown-word'}, 'count': 398},
 {'_id': {'source': 'autobrowsing-mars-distil'}, 'count': 1053},
 {'_id': {'source': 'evalstyle_question'}, 'count': 7679},
 {'_id': {'source': 'CodeAct'}, 'count': 437},
 {'_id': {'source': 'clova_x_seed_input_1064'}, 'count': 1034},
 {'_id': {'source': 'ab-ci-hybrid'}, 'count': 54},
 {'_id': {'source': 'autobrowsing-hcd'}, 'count': 1843},
 {'_id': {'source': 'tooluse-negative'}, 'count': 105

In [9]:
write_collection = db["analyze-240718-groupby-source"]

write_collection.insert_many(collection.aggregate(pipeline))

InsertManyResult([{'source': 'backjun_round2'}, {'source': 'ci_seed'}, {'source': 'aihub.150.numeric_mr'}, {'source': 'autobrowsing'}, {'source': 'programmers'}, {'source': 'aihub.71565.tableqa'}, {'source': 'python_execution'}, {'source': 'autobrowsing-mars-distil'}, {'source': 'evalstyle_question'}, {'source': 'unknown-word'}, {'source': 'chart-to-table'}, {'source': '240613_error_correction'}, {'source': 'tooluse-negative'}, {'source': 'OpenMathInstruct-1'}, {'source': 'CodeAct'}, {'source': 'clova_x_seed_input_1064'}, {'source': 'ab-ci-hybrid'}, {'source': 'autobrowsing-hcd'}, {'source': 'autobrowsing-tooluse'}, {'source': 'programmers_round2'}, {'source': 'samsung_sw_final'}, {'source': 'autobrowsing-goldenset'}, {'source': 'speaker.calendar.case1'}, {'source': 'user_agent'}, {'source': 'backjun'}, {'source': 'tableQA'}, {'source': 'autobrowsing-new-300'}, {'source': 'redteam-tooluse-augmented'}, {'source': 'evalstyle_question_2'}, {'source': 'wtq'}], acknowledged=True)

In [50]:
# 2.
result = list(collection.find(
{
    'data': {
        '$elemMatch': {
            'content': {
                '$regex': '.*<<ImageDisplayed>>.*'
            }
        }
    }
}))


write_collection = db["analyze-240718-image-displayed-turns"]
for doc in result:
    traceback_indices = []
    for i, turn in enumerate(doc["data"]):
        if "content" in turn and turn["content"].find("<<ImageDisplayed>>") != -1:
            traceback_indices.append(i)
    doc["target_turn_idx"] = traceback_indices
    write_collection.insert_one(doc)
    

In [60]:
# 3.
result = list(collection.find(
{
    'data': {
        '$elemMatch': {
            'content': {
                '$regex': '.*Traceback \(most recent call last\).*'
            }
        }
    }
}))


write_collection = db["analyze-240718-python-traceback-turns"]
for doc in result:
    traceback_indices = []
    for i, turn in enumerate(doc["data"]):
        if "content" in turn and turn["content"].find("Traceback (most recent call last)") != -1:
            traceback_indices.append(i)
    doc["target_turn_idx"] = traceback_indices
    write_collection.insert_one(doc)
    